# Vector Database Basics

Vector databases help us store, manage, and query the embeddings we created for generative AI, recommenders, and search engines.

Across many of the common use cases, users often find that they need to manage more than just vectors.
To make it easier for practitioners, vector databases should store and manage all of the data they need:
- embedding vectors
- categorical metadata
- numerical metadata
- timeseries metadata
- text / pdf / images / video / point clouds

And support a wide range of query workloads:
- Vector search (may require ANN-index)
- Keyword search (requires full text search index)
- SQL (for filtering)

For this exercise we'll use LanceDB since it's open source and easy to setup

In [1]:
# pip install -U --quiet lancedb pandas pydantic [This has been pre-installed for you]

## Creating tables and adding data

Let's create a LanceDB table called `cats_and_dogs` under the local database directory `~/.lancedb`.
This table should have 4 fields:
- the embedding vector
- a string field indicating the species (either "cat" or "dog")
- the breed
- average weight in pounds

We're going to use pydantic to make this easier. First let's create a pydantic model with those fields

In [2]:
from lancedb.pydantic import vector, LanceModel

class CatsAndDogs(LanceModel):
    vector: vector(2)
    species: str
    breed: str
    weight: float

Now connect to a local db at ~/.lancedb and create an empty LanceDB table called "cats_and_dogs"

In [3]:
import lancedb

db = lancedb.connect("~/.lancedb")
table_name = "cats_and_dogs"
db.drop_table(table_name, ignore_missing=True)
table = db.create_table(table_name, schema=CatsAndDogs)

Let's add some data

First some cats

In [4]:
data = [
    CatsAndDogs(
        vector=[1., 0.],
        species="cat",
        breed="shorthair",
        weight=12.,
    ),
    CatsAndDogs(
        vector=[-1., 0.],
        species="cat",
        breed="himalayan",
        weight=9.5,
    ),
]

Now call the `LanceTable.add` API to insert these two records into the table

In [5]:
table.add([dict(d) for d in data])

Let's preview the data

In [6]:
table.head().to_pandas()

,vector,species,breed,weight
0,"[1.0, 0.0]",cat,shorthair,12.0
1,"[-1.0, 0.0]",cat,himalayan,9.5


Now let's add some dogs

In [7]:
data = [
    CatsAndDogs(
        vector=[0., 10.],
        species="dog",
        breed="samoyed",
        weight=47.5,
    ),
    CatsAndDogs(
        vector=[0, -1.],
        species="dog",
        breed="corgi",
        weight=26.,
    )
]

In [8]:
table.add([dict(d) for d in data])

In [9]:
table.head().to_pandas()

,vector,species,breed,weight
0,"[1.0, 0.0]",cat,shorthair,12.0
1,"[-1.0, 0.0]",cat,himalayan,9.5
2,"[0.0, 10.0]",dog,samoyed,47.5
3,"[0.0, -1.0]",dog,corgi,26.0


## Querying tables

Vector databases allow us to retrieve data for generative AI applications. Let's see how that's done.

Let's say we have a new animal that has embedding [10.5, 10.], what would you expect the most similar animal will be?
Can you use the table we created above to answer the question?

**HINT** you'll need to use the `search` API for LanceTable and `limit` / `to_df` APIs. For examples you can refer to [LanceDB documentation](https://lancedb.github.io/lancedb/basic/#how-to-search-for-approximate-nearest-neighbors).

**SOLUTION** if you answered "samoyed" then you're right!. Here we pass the vector into `search` and make a chained call to `limit` with 1 as the param. Then we'll need to call `to_df` to execute the query and convert the results to a pandas dataframe. In addition to the data columns, you'll also see a "score" column, which contains the distance score between the query vector and the returned vector. In this case, the score is the square of the Euclidean distance.

In [10]:
table.search([10.5, 10.,]).limit(1).to_df()

,vector,species,breed,weight,_distance
0,"[0.0, 10.0]",dog,samoyed,47.5,110.25


Now what if we use cosine distance instead? Would you expect that we get the same answer? Why or why not?

**HINT** you can add a call to `metric` in the call chain

**SOLUTION** Remember that cosine distance is the angle between vectors. So while the query vector [10.5, 10.] is closer in Euclidean distance to Samoyed, the angle is slightly closer to shorthair cats.

In [11]:
table.search([10.5, 10.]).metric("cosine").limit(1).to_df()

,vector,species,breed,weight,_distance
0,"[1.0, 0.0]",cat,shorthair,12.0,0.275862


## Filtering tables

In practice, we often need to specify more than just a search vector for good quality retrieval. Oftentimes we need to filter the metadata as well.

Please write code to retrieve two most similar examples to the embedding [10.5, 10.] but only show the results that is a cat.

**HINT** In LanceDB, for additional filtering, you can add a call to `where` in the call chain and pass in a SQL-like filter string.

In [12]:
table.search([10.5, 10.,]).limit(2).where("species='cat'").to_df()

,vector,species,breed,weight,_distance
0,"[1.0, 0.0]",cat,shorthair,12.0,190.25


## Creating ANN indices

For larger tables (e.g., >1M rows), searching through all of the vectors becomes quite slow. Here is where the Approximate Nearest Neighbor (ANN) index comes into play. While there are many different ANN indexing algorithms, they all have the same purpose - to drastically limit the search space as much as possible while losing as little accuracy as possible

For this problem we will create an ANN index on a LanceDB table and see how that impacts performance

### First let's create some data

Given the constraints of the classroom workspace, we'll complete this exercise by creating 100,000 vectors with 16D in a new table. Here the embedding values don't matter, so we simply generate random embeddings as a 2D numpy array. We then use the `vec_to_table` function to convert that in to an Arrow table, which can then be added to the table.

In [13]:
from lance.vector import vec_to_table
import numpy as np

mat = np.random.randn(100_000, 16)
table_name = "exercise3_ann"
db.drop_table(table_name, ignore_missing=True)
table = db.create_table(table_name, vec_to_table(mat))

### Let's establish a baseline without an index

Before we create the index, let's make sure know what we need to compare against.

We'll generate a random query vector and record it's value in the `query` variable so we can use the same query vector with and without the ANN index.

In [14]:
query = np.random.randn(16)
table.search(query).limit(10).to_df()

,vector,_distance
0,"[0.40637052, -0.6848565, 0.25305176, 0.1160183...",4.934102
1,"[-1.0806783, -0.37886325, -0.43526447, 0.80882...",4.939671
2,"[-0.7834152, 0.22190379, 0.62918437, 0.5440492...",5.026000
3,"[-0.85893506, -0.82675505, -0.27404046, 0.0690...",5.045324
4,"[-0.0692185, 0.09582594, 0.110043034, 0.870683...",5.185118
5,"[0.4731883, -0.43243143, 0.11517821, 0.5844016...",5.280421
6,"[-0.13414739, 0.7341682, -0.7966965, 1.3163494...",5.283449
7,"[-0.7620548, -0.21515357, -0.18533202, 0.35712...",5.439384
8,"[0.20514311, -0.7195129, 0.083456345, -0.19420...",5.657211
9,"[0.027963892, 0.063259386, -0.67383015, 0.7698...",5.827345


Please write code to compute the average latency of this query

**SOLUTION** There are several possible solutions. Given that we're in a notebook environment, the easiest is probably using the %timeit magic function to run the command a bunch of times and compute the average

In [15]:
%timeit table.search(np.random.randn(16)).limit(10).to_arrow();

19.9 ms ± 742 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Now let's create an index

There are many possible index types ranging from hash based to tree based to partition based to graph based.
For this task, we'll create an IVFPQ index (partition-based index with product quantization compression) using LanceDB.

Please create an IVFPQ index on the LanceDB table such that each partition is 4000 rows and each PQ subvector is 8D.

**HINT** 
1. Total vectors / number of partitions = number of vectors in each partition
2. Total dimensions / number of subvectors = number of dimensions in each subvector

In [16]:
table.create_index(num_partitions=16, num_sub_vectors=8)

Sample 4096 out of 65536 to train kmeans of 16 dim, 16 clusters


Now let's search through the data again. Notice how the answers now appear different.
This is because an ANN index is always a tradeoff between latency and accuracy.

In [17]:
table.search(query).limit(10).to_df()

,vector,_distance
0,"[-1.0806783, -0.37886325, -0.43526447, 0.80882...",4.783734
1,"[-0.85893506, -0.82675505, -0.27404046, 0.0690...",4.882057
2,"[0.40637052, -0.6848565, 0.25305176, 0.1160183...",4.941090
3,"[0.4731883, -0.43243143, 0.11517821, 0.5844016...",5.062208
4,"[-0.0692185, 0.09582594, 0.110043034, 0.870683...",5.200292
5,"[-0.13414739, 0.7341682, -0.7966965, 1.3163494...",5.321503
6,"[-0.7834152, 0.22190379, 0.62918437, 0.5440492...",5.378174
7,"[-0.7620548, -0.21515357, -0.18533202, 0.35712...",5.437410
8,"[0.20514311, -0.7195129, 0.083456345, -0.19420...",5.497247
9,"[-0.66275835, -0.77101654, -0.7916006, 0.02394...",5.500407


Now write code to compute the average latency for querying the same table using the ANN index.

**SOLUTION** The index is implementation detail, so it should just be running the same code as above. You should see almost an order of magnitude speed-up. On larger datasets, this performance difference should be even more pronounced.

In [18]:
%timeit table.search(np.random.randn(16)).limit(10).to_arrow();

9.51 ms ± 344 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Deleting rows

Like with other kinds of databases, you should be able to remove rows from the table.
Let's go back to our tables of cats and dogs

In [19]:
table = db["cats_and_dogs"]

In [20]:
len(table)

4

Can you use the `delete` API to remove all of the cats from the table?

**HINT** use a SQL like filter string to specify which rows to delete from the table

In [21]:
table.delete("species='cat'")

In [22]:
len(table)

2

## What if I messed up?

Errors is a common occurrence in AI. What's hard about errors in vector search is that oftentimes a bad vector doesn't cause a crash but just creates non-sensical answers. So to be able to rollback the state of the database is very important for debugging and reproducibility

So far we've accumulated 4 actions on the table:
1. creation of the table
2. added cats
3. added dogs
4. deleted cats

What if you realized that you should have deleted the dogs instead of the cats?

Here we can see the 4 versions that correspond to the 4 actions we've done

In [23]:
table.list_versions()

[{'version': 1,
  'timestamp': datetime.datetime(2023, 8, 30, 14, 26, 27, 208330),
  'metadata': {}},
 {'version': 2,
  'timestamp': datetime.datetime(2023, 8, 30, 14, 26, 27, 236584),
  'metadata': {}},
 {'version': 3,
  'timestamp': datetime.datetime(2023, 8, 30, 14, 26, 27, 294913),
  'metadata': {}},
 {'version': 4,
  'timestamp': datetime.datetime(2023, 8, 30, 14, 33, 50, 823708),
  'metadata': {}}]

Please write code to restore the version still containing the whole dataset

In [24]:
table = db["cats_and_dogs"]

In [25]:
len(table)

2

In [26]:
table.restore(3)

In [27]:
table.delete("species='dog'")

In [28]:
table.list_versions()

[{'version': 1,
  'timestamp': datetime.datetime(2023, 8, 30, 14, 26, 27, 208330),
  'metadata': {}},
 {'version': 2,
  'timestamp': datetime.datetime(2023, 8, 30, 14, 26, 27, 236584),
  'metadata': {}},
 {'version': 3,
  'timestamp': datetime.datetime(2023, 8, 30, 14, 26, 27, 294913),
  'metadata': {}},
 {'version': 4,
  'timestamp': datetime.datetime(2023, 8, 30, 14, 33, 50, 823708),
  'metadata': {}},
 {'version': 5,
  'timestamp': datetime.datetime(2023, 8, 30, 14, 33, 50, 861895),
  'metadata': {}},
 {'version': 6,
  'timestamp': datetime.datetime(2023, 8, 30, 14, 33, 50, 903319),
  'metadata': {}}]

In [29]:
table.to_pandas()

,vector,species,breed,weight
0,"[1.0, 0.0]",cat,shorthair,12.0
1,"[-1.0, 0.0]",cat,himalayan,9.5


## Dropping a table

You can also choose to drop a table, which also completely removes the data.
Note that this operation is not reversible.

In [30]:
"cats_and_dogs" in db

True

Write code to irrevocably remove the table "cats_and_dogs" from the database

In [31]:
db.drop_table("cats_and_dogs")

How would you verify that the table has indeed been deleted?

In [32]:
table.name in db

False

## Summary

Congrats, in this exercise you've learned the basic operations of vector databases from creating tables, to adding data, and to querying the data. You've learned how to create indices and you saw first hand how it changes the performance and the accuracy. Lastly, you've learned how to debug and rollback when errors happen.